In [2]:
# Load Required Libraries
import os
import sys
import glob
import calendar

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import  mapping #shape, 
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
# from contextlib import contextmanager

from osgeo import gdal #, osr, ogr


import matplotlib.pyplot as plt

In [19]:
# Load Functions defined in functions4stepFunction jupyter notebook

%run functions4stepFunction.ipynb

In [10]:
# Define Annual and Pernnial crops

allcrops = {'Barley':'Bar', 'Cassava':'Cas', 'Cotton':'Cot', 'Fodder':'Fod', 'Groundnuts':'Gro',
            'Maize':'Maz', 'Millet':'Mil', 'Oil palm':'Olp', 'Others annual':'Otha', 'Others perennial':'Othp',
            'Potatoes':'Pot', 'Pulses':'Pul', 'Rapeseed':'Rap', 'Rice':'Ric', 'Rye':'Rye', 'Sorghum':'Sor',
            'Soybeans':'Soy', 'Coffee':'Stm', 'Cocoa':'Stm', 'Sugar beet':'Sub', 'Sugar cane':'Sug', 'Sunflower':'Sun', 
            'Wheat':'Whe'}
Annual = ("Barley", "Cassava", "Groundnuts","Maize","Millet",
          "Potatoes","Pulses","Rapeseed","Rice1","Rice2",
          "Sorghum","Soybeans","Sunflower","Wheat1","Wheat2",
          "Cotton", "Fodder", "Others annual", "Rye", "Sugar beet")

Perennial = ("Sugarcane", "Citrus", "Grape",  "Cocoa", "Coffee",
             "Oil palm", "Date palm", "Others perennial")


In [11]:
# Load the datasets
# Root folder for saving rasters clipped by unitcode
root_folder = "../scratch"
shp_00 = "../data/admin_boundaries/merged_2000.shp"
shp_15 = "../data/admin_boundaries/merged_2015.shp"
scratch_folder = "../scratch"

# Load the crop and area data clipped by unit code 
raster_folder = "../data/Ras_shape_out/Rasters"
tiff_files = glob.glob("{}/**/*.tif".format(raster_folder), recursive=True)
tiff_files


# Folder with crop calendar data
CropCal_dir = "../data/CropCalender"

# Load crop calendar data and
# Rank each by crop area in each unit code
ranked_df_ir = {}
ranked_df_rf = {}

for year in ["00", "05", "10", "15"]: 
  # Irrigated
  ranked_df_ir[year] =  rankbyarea(CropCal_dir = CropCal_dir,
                                  irrigated_rainfed = "ir", year=year)
  # Rainfed
  ranked_df_rf[year] = rankbyarea(CropCal_dir = CropCal_dir, 
                        irrigated_rainfed = "rf", year=year,
                         by_state = True)
    


# ranked_df_rf["15"]
# ranked_df_rf["00"]
ranked_df_ir["00"]

../data/CropCalender/MIRCA-OS_2000_ir.csv
../data/CropCalender/MIRCA-OS_2000_rf.csv
../data/CropCalender/MIRCA-OS_2005_ir.csv
../data/CropCalender/MIRCA-OS_2005_rf.csv
../data/CropCalender/MIRCA-OS_2010_ir.csv
../data/CropCalender/MIRCA-OS_2010_rf.csv
../data/CropCalender/MIRCA-OS_2015_ir.csv
../data/CropCalender/MIRCA-OS_2015_rf.csv


,rank,Country,State,District,unit_code,Crop,Subcrop,Type,Growing_area,Planting_Month,Maturity_Month
0,1,Afghanistan,Afghanistan,Afghanistan,40000,Wheat1,1,Annual,856749,11,5
1,2,Afghanistan,Afghanistan,Afghanistan,40000,Maize,1,Annual,239997,6,9
2,3,Afghanistan,Afghanistan,Afghanistan,40000,Fodder,1,Fodder,154188,1,12
3,4,Afghanistan,Afghanistan,Afghanistan,40000,Rice1,1,Annual,128667,4,10
4,5,Afghanistan,Afghanistan,Afghanistan,40000,Barley,1,Annual,124691,12,4
...,...,...,...,...,...,...,...,...,...,...,...
119764,22,Zambia,Zambia,Zambia,8940000,Millet,1,Annual,0,5,9
119765,23,Zambia,Zambia,Zambia,8940000,Groundnuts,1,Annual,0,5,9
119766,24,Zambia,Zambia,Zambia,8940000,Fodder,1,Fodder,0,1,12
119767,25,Zambia,Zambia,Zambia,8940000,Cocoa,1,Perennial,0,1,12


In [5]:
# # Clip the rasters 

# root_folder_subfolders = {y:"{}/{}".format(root_folder, y) for y in ["00", "05", "10", "15"]}

# for year in ["00", "05", "10", "15"]:
#   shp_path = shp_15 if year in ["15"] else shp_00
#   raster_main_dir = "../data/cropRaster/{}{}".format("20", year)
#   output_dir = "{}/Clipped".format(scratch_folder)

#   raster_paths = glob.glob("{}/*.tif".format(raster_main_dir), recursive=False)
#   for raster_path in raster_paths:
#     mask_raster_by_geometries(shp_path, raster_path, scratch_folder, 
#                               year,  visualize=False)



In [ ]:
for year in ["00", "05", "10", "15"]:
    """
    Loop through the specified years and perform the MIRCA steps for each year.

    This loop iterates through four years, loading data, processing each unit code, 
    and performing the MIRCA steps sequentially. For each year:
    - Check if the irrigated data is available for the year.
    - Load raster data for AEI, Ara, and CLE.
    - Perform MIRCA steps 1 to 7 for irrigated and rainfed crops.

    Parameters:
    ----------
    year : str
        The last two digits of the year being processed (e.g., "00" for 2000).
    
    Notes:
    -----
    The function continues to the next year if the irrigated data is not available.
    """

    if ranked_df_ir[year] is None:
        print("ranked_df_ir {} is not available".format(year))
        continue

    dist_array = {}
    irri_totals = {}

    # Get all unique UNIT CODEs
    # basename_unit_codes = [
    #     os.path.splitext(os.path.basename(tiff_file))[0]
    #     for tiff_file in glob.glob("{}/*/**.tif".format(root_folder), recursive=True)
    # ]
    basename_unit_codes = np.unique(ranked_df_ir[year].unit_code)
    unit_codes = np.unique(np.array(basename_unit_codes))

    for unit_code in unit_codes:
        # print(unit_code) 
        # if unit_code == 500060:
        #     break

        # Get the crop-specific irrigated and rainfed growing area and crop calendar of the unit code district
        ranked_df_ir_unitcode = ranked_df_ir[year].loc[(ranked_df_ir[year]['unit_code'] == int(unit_code))]
        ranked_df_rf_unitcode = ranked_df_rf[year].loc[(ranked_df_rf[year]['unit_code'] == int(unit_code))]

        # Skip unitcodes not in irrigated ranked data
        if not ranked_df_ir_unitcode.shape[0]:
            continue

        # LOAD THE AEI, Ara, AND CLE TIFF files for the unit code
        skip_outer = False
        for aac in ["AEI", "Ara", "CLE"]:
            tiff_file_aac = f"{root_folder}/{year}/{aac}_{year}/{unit_code}.tif"
            
            if not os.path.isfile(tiff_file_aac):
                # print(f"Missing {tiff_file_aac}")
                skip_outer = True
                continue
            dist_aac = gdal.Open(tiff_file_aac, gdal.GA_ReadOnly).ReadAsArray()
            dist_aac[dist_aac < 0] = 0  
            dist_array["dist_array_{}".format(aac.lower())] = dist_aac
        if  skip_outer:
            continue

        # print(unit_code) 
        # Loop through the crops by rank while allocating area in the seven steps
        for j in range(1, int(np.max(ranked_df_ir_unitcode['rank']) + 1)):
            ranked_df_ir_rank = ranked_df_ir_unitcode.loc[(ranked_df_ir_unitcode['rank'] == j)]

            # Extract array data for crop type and assign to corresponding variable in crop_dict
            crop_name = ranked_df_ir_rank.Crop.tolist()[0]

            # Get the path to the crop's TIFF file
            tiff_file = getpath2croptiff(crop_name, root_folder, allcrops, year, unit_code)
            
            # Skip the row if crop not in all crops
            if not tiff_file:
                print("Crop {} is not in allcrops dictionary".format(crop_name))
                continue


            print(tiff_file)
            AH_array = gdal.Open(tiff_file, gdal.GA_ReadOnly).ReadAsArray()
            AH_array[AH_array < 0] = 0  

            # Perform the MIRCA steps one at a time with the data
            step1 = step00(step=1, year=year, ranked_df_ir_rank=ranked_df_ir_rank, 
                           ranked_df_rf_rank=None,
                           dist_array=dist_array, AH_array=AH_array,
                           df_tot=None, ATS_2=None, AD_area=None, df_cumulative=None,
                           df_mon=None, TAD_12=None)
            print(np.sum(step1['AD_area']))

            step2 = step00(step=2, year=year, ranked_df_ir_rank=ranked_df_ir_rank, 
                           ranked_df_rf_rank=None,
                           dist_array=dist_array, AH_array=AH_array,
                           df_tot=step1['df_tot'], ATS_2=step1['ATS_2'], AD_area=step1['AD_area'],
                           df_cumulative=step1['df_cumulative'], df_mon=step1['df_mon'],
                           c_s=step1['c_s'], r_s=step1['r_s'],
                           TAD_12=None)

            print(np.sum(step2['AD_area']))
            step3 = step00(step=3, year=year, ranked_df_ir_rank=ranked_df_ir_rank, 
                           ranked_df_rf_rank=None,
                           dist_array=dist_array, AH_array=AH_array,
                           df_tot=step2['df_tot'], ATS_2=step2['ATS_2'], AD_area=step2['AD_area'], 
                           df_cumulative=step2['df_cumulative'], df_mon=step2['df_mon'],
                           c_s=step2['c_s'], r_s=step2['r_s'],
                           TAD_12=None)

            print(np.sum(step3['AD_area']))
            # Check if the crop is in both irrigated and rainfed
            step4 = step00(step=4, year=year, ranked_df_ir_rank=ranked_df_ir_rank, 
                           ranked_df_rf_rank=None,
                           dist_array=dist_array, AH_array=AH_array,
                           df_tot=step3['df_tot'], ATS_2=step3['ATS_2'], AD_area=step3['AD_area'],
                           df_cumulative=step3['df_cumulative'], df_mon=step3['df_mon'],
                           c_s=step3['c_s'], r_s=step3['r_s'],
                           TAD_12=None,
                           template_tiff=tiff_file)

            print(np.sum(step4['AD_area']))
        # Ensure necessary arrays are loaded
        array0 = [dist_array["dist_array_ara"], dist_array["dist_array_aei"], dist_array["dist_array_cle"]]
        if any(element is None for element in array0):
            break
        # if np.sum(step4['AD_area'] <= 0:
        #     break

        # Skip unitcode if there are no crops
        if not ranked_df_ir_unitcode.shape[0]:
            continue

        for j in range(1, int(np.max(ranked_df_rf_unitcode['rank']) + 1)):
            ranked_df_rf_rank = ranked_df_rf_unitcode.loc[(ranked_df_rf_unitcode['rank'] == j)]
            
            crop_name = ranked_df_rf_rank.Crop.tolist()[0]
            tiff_file = getpath2croptiff(crop_name, root_folder, allcrops, year, unit_code)
            
            if not tiff_file:
                # print("Crop {} is not in allcrops dictionary".format(crop_name))
                continue

            step5 = step00(step=5, year=year, ranked_df_ir_rank=ranked_df_ir_rank, 
                           ranked_df_rf_rank=ranked_df_rf_rank,
                           dist_array=dist_array, AH_array=AH_array,
                           df_tot=step4['df_tot'], ATS_2=step4['ATS_2'], AD_area=step4['AD_area'],
                           df_cumulative=step4['df_cumulative'], df_mon=step4['df_mon'],
                           c_s=step4['c_s'], r_s=step4['r_s'],
                           TAD_12=None,
                           template_tiff=tiff_file)

            print(np.sum(step5['AD_area']))
            step6 = step00(step=6, year=year, ranked_df_ir_rank=ranked_df_ir_rank, 
                           ranked_df_rf_rank=ranked_df_rf_rank,
                           dist_array=dist_array, AH_array=AH_array,
                           df_tot=step5['df_tot'], ATS_2=step5['ATS_2'], AD_area=step5['AD_area'],
                           df_cumulative=step5['df_cumulative'], df_mon=step5['df_mon'],
                           c_s=step5['c_s'], r_s=step5['r_s'],
                           TAD_12=None,
                           template_tiff=tiff_file)

            print(np.sum(step6['AD_area']))
            step7 = step00(step=7, year=year, ranked_df_ir_rank=ranked_df_ir_rank, 
                           ranked_df_rf_rank=ranked_df_rf_rank,
                           dist_array=dist_array, AH_array=AH_array,
                           df_tot=step6['df_tot'], ATS_2=step6['ATS_2'], AD_area=step6['AD_area'],
                           df_cumulative=step6['df_cumulative'], df_mon=step6['df_mon'],
                           c_s=step6['c_s'], r_s=step6['r_s'],
                           TAD_12=None,
                           template_tiff=tiff_file)

            print(np.sum(step7['AD_area']))

../scratch/00/Bar_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Cas_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Whe_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Sun_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Sug_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Sub_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Soy_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Sor_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Ric_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area distributed after step 4: 0
0.0
../scratch/00/Ric_00/3560001.tif
0.0
0.0
0.0
Iteration completed.
Area di